In [1]:
import requests
import json
import pandas as pd
import datetime 
from datetime import datetime as dt
import pytz
import time
import re

# intro 

This notebook is part of the construction of the Github Activity Lego , it contains the construction of a module that : 

- accepts a series of github projects URL; 

- uses the github API to get data about the  project interactions given a specific timeframe ( here we used the first week of  Gitcoin Grants Round 15 and 6 months back from that week. So, from week 14 till week 38 of 2022).

- Returns 3 dataframes by the end:
     - github_addittions: additions pushed to the repo by week for each of the repo
     
     - github_deletions: deletions pushed to the repo by week for each of the repo
     
	 - raw dataframe : containing the repo information, the status code of the extraction of data form the Github API and the json with the whole data for that repo
     
### How to use it: 

It will be used on the extraction of grantees project data to investigate fraudulent behavior and fake repos on grants subscriptions. 

- [here's a notebook with exploratory data analysis on the GR15 applicants repos using this module](https://github.com/stefi-says/web3_projects_and_contributions/blob/3f0cd1706ac061143118a2c959ee1fcb538a9fa5/Scripts%20and%20tools/github%20activity%20lego%20study.ipynb)

The way it is now this module can be used to produce training datasets, exploratory data analysis or enhance information about projects for human analysts for example:   


- making a list of projects that claims to use github and extract their information so humana analysts can compare the maturity off that project, how long it have been around and their general activity logs



### Some important information and further development:

- You will need a PAT ( personal authorization token) or other authorization key, for more information , check this page of the [documentation](https://docs.github.com/en/rest/guides/getting-started-with-the-rest-api?apiVersion=2022-11-28).  


-  Even though the PAT was added to it,  it was only possible to make 60 calls an hour , it's our desire to figure out how to amplify that .  



- Be aware that the APi does not delivers data to deleted or empty repo, so:
    - by the end of the process the non existent repo will show on the raw dataframe but with "extrat_status_code" =  404 and a json containing the error explanation on "repo_data". 
    - the empty repo will show 'status_code' = 204 and an empty dictionary on "repo_data".  
    
    

- The code is built to break if you reach the rate limit and receives back the "status_code" = 403, but it seems to be not respecting that and runs forever. That's another improvement to be made.  We advise to break the functions in a script and run it in small batches. 
 

### License 
It's Open  Source and was built by stefi_says#1654 and Gray#3751, and follows teh whole project license: 

Apache License 2.0 

Please reach us on discord for any doubt 


# importing data - example

In [2]:
grants = pd.read_excel('gr15_grants.xlsx')

gr_aplic = pd.read_json('grants_applications_gr15.json').T

df  = gr_aplic.merge(grants, on = 'grant_id' )

In [3]:
df.columns

Index(['grant_id', 'active_x', 'approved', 'address_x', 'title_x', 'url',
       'description_x', 'created_on_x', 'active_y', 'title_y', 'address_y',
       'amount_received', 'amount_received_in_round', 'contribution_count',
       'contributor_count', 'description_y', 'website', 'github_project_url',
       'twitter_handle_2', 'twitter_handle_1', 'twitter_verified',
       'created_on_y', 'last_update'],
      dtype='object')

In [4]:
df.shape

(403, 23)

# functions

In [5]:

# getting the repo data , returning a json with the additions, deletions and week a timestamps ( Sundays)

def github_code_stats( owner, repo, authorization_token):
    url = "https://api.github.com/repos/{owner}/{repo}/stats/code_frequency"
    headers = {
     'X-GitHub-Api-Version': '2022-11-28', 
     'accept':'application/vnd.github+json', 
     'Authorization': authorization_token
     }
    
    response = requests.get(url.format(owner=owner, repo=repo), headers=headers)
    response_status = response.status_code
    response_json = {}
    if (
        response.status_code != 204 and
        response.headers["content-type"].strip().startswith("application/json")
    ):
        try:
            response_json  = response.json()

        except ValueError:
            pass

    return {'status_code': response_status ,'repo_data' : response_json}


#-----------------
# get the owner and the repo name of a list of projects github urls
# accepts URLS give back a df with URL / owner of the repo / Repo name
# does not work with None values
#function return a df

def get_owner_repo(github_urls):

    owner = []
    repo = []
    url = []

    for i in github_urls:
        matches = re.search(r"github\.com\/([\w\-\.]+)\/([\w\-\.]+)", i)
        if matches:
            username = matches.group(1)
            repository_name = matches.group(2)
            url.append(i)
            owner.append(username)
            repo.append(repository_name)
        else:
            matches = re.search(r"github\.com\/([\w\-\.]+)", i)
            if matches:
                owner_name = matches.group(1)
                url.append(i)
                owner.append(owner_name)
                repo.append(None)
            else:
                url.append(i)
                owner.append(None)
                repo.append(None)

    github_owner_repo = pd.DataFrame(data= {'url': url, 'owner': owner, 'repo': repo})
    return github_owner_repo
    
#------------------------
# receives the df produced on the last function
# makes a first call for every line 
# saves the status code and the json returned
# as long we have 202 as status  code ( github is still gathering data for that repo)
# it waits and makes calls till it gets '200' and the repo data
# return a df with the columns =  ['url', 'owner', 'repo', 'extract_status_code', 'repo_data']

def retrive_git_data(owner_repo_names, authorization_token):
    
        column_names = ['url', 'owner', 'repo', 'extract_status_code', 'repo_data']
        git_data  = pd.DataFrame([],columns = column_names)

        for i in range(0,len(owner_repo_names['owner'])):
                # gettin owner and repo
                git_owner = owner_repo_names.iloc[i]['owner']
                git_repo = owner_repo_names.iloc[i]['repo']

                # pocking the APi to start gathering the stats
                git_extract = github_code_stats(git_owner, git_repo, authorization_token)
                
                data = [{'url' : owner_repo_names.iloc[i]['url'], 'owner' : git_owner, 'repo' : git_repo,  'extract_status_code': 
                        git_extract['status_code'] , 'repo_data': git_extract['repo_data']}]
                df = pd.DataFrame(data = data)
                
                git_data = pd.concat([git_data, df])


        git_data.set_index('url', inplace = True)

        while git_data['extract_status_code'].isin([202]).sum() != 0:

            time.sleep(20)

            redo_df = git_data[git_data['extract_status_code'] == 202].reset_index().copy()
            for i in range(len(redo_df)):

                redo_owner = redo_df.loc[i, 'owner']
                redo_repo = redo_df.loc[i, 'repo']

                git_extract = github_code_stats(redo_owner, redo_repo, authorization_token)

                if (git_extract['status_code'] != 403):

                    git_data.at[redo_df['url'][i], 'extract_status_code'] = git_extract['status_code']
                    git_data.at[redo_df['url'][i], 'repo_data'] = git_extract['repo_data']

                else:

                    break


        return git_data

#---------------------------------------------------------------------
# function used inside 'timeframing_data' to get the time stamp of the sunday of a given week by its number
# its used to filter the start and finish of the period to colect the data 

def sunday_timestamp(week_number, year):
    # Create a datetime object for the first day of the given year
    first_day = datetime.datetime(year, 1, 1, tzinfo=pytz.utc)
    
    # Calculate the number of days to the first Sunday of the year
    days_to_first_sunday = (6 - first_day.weekday()) % 7
    
    # Calculate the number of days to the Sunday of the given week
    days_to_sunday = (week_number - 1) * 7 + days_to_first_sunday
    
    # Create a datetime object for the Sunday of the given week
    sunday = first_day + datetime.timedelta(days=days_to_sunday)
    
    # Convert the datetime object to a UTC timestamp
    return int(sunday.timestamp())


#-------------------------------------------------------
# function to treat the json data generated by github_code_stats returns a datafram with the url/weeks/ additons or deletion per week on that repo
# repo_data must be in json 
# start and end date aggregation in week number 
# year number like 'yyyy' = '2023'

def tretened_df(raw_git_data, start_date_aggregation, end_date_aggregation, year_to_start,year_to_finish ):

    columns_dates = [] 

    for w in range(end_date_aggregation, start_date_aggregation +1): # prestar atenção na questão do inclusivo exclusivo
        columns_dates.append(dt.date(dt.fromtimestamp(sunday_timestamp(w ,year_to_start))))

    addtions_df  = pd.DataFrame([], columns = columns_dates)


    deletions_df  = pd.DataFrame([], columns = columns_dates)

    valid_git_data = raw_git_data[raw_git_data['extract_status_code'] == 200]

    valid_git_data

    for n in range(0,(valid_git_data.shape[0])):  #" a questão tava aqui no "n'""
        weeks = []
        addition = []
        deletions = []

        for i in range(0,(len(valid_git_data['repo_data'][n])-1)):

            weeks.append(dt.date(dt.fromtimestamp(valid_git_data['repo_data'][n][i][0])))
            addition.append(valid_git_data['repo_data'][n][i][1])
            deletions.append(valid_git_data['repo_data'][n][i][2])


        week_addition = pd.DataFrame( data = [weeks, addition, deletions]).T

        week_addition.columns = ['weeks', 'addition', 'deletions']

        additions_by_week = week_addition[(week_addition['weeks']<= dt.date(dt.fromtimestamp(sunday_timestamp(start_date_aggregation,year_to_start))))
                                           & (week_addition['weeks'] >=  dt.date(dt.fromtimestamp(sunday_timestamp(end_date_aggregation,year_to_start))))]  


        ad_df = additions_by_week[['weeks','addition']].T
        ad_df.columns=ad_df.iloc[0] 
        ad_df.drop(labels='weeks', inplace = True)
        ad_df.rename(index = {'addition' :valid_git_data.index[n]}, inplace = True)
        addtions_df= pd.concat([addtions_df, ad_df])

        del_df = additions_by_week[['weeks','deletions']].T
        del_df.columns=del_df.iloc[0] 
        del_df.drop(labels='weeks', inplace = True)
        del_df.rename(index = {'deletions' :valid_git_data.index[n]}, inplace = True)
        deletions_df= pd.concat([deletions_df, del_df])

    return addtions_df, deletions_df

#--------------------------------------------


#########################################################final compile function #####################################


def repo_additions_deletion(url_repo_series,start_date_aggregation,end_date_aggregation,year_to_start, year_to_finish, authorization_token ):
    # from a list get the owner and repo names
    owner_repo_names  = get_owner_repo(url_repo_series)

    #cleaning the none values, dealing with index issues
    owner_repo_names = owner_repo_names[~owner_repo_names['repo'].isna()]
    owner_repo_names = owner_repo_names[owner_repo_names['repo'] != '']
    owner_repo_names = owner_repo_names.drop(owner_repo_names[owner_repo_names.duplicated()].index)
    owner_repo_names.reset_index(drop = True, inplace = True)


    #  pocking , waiting and getting the data from the API. 
    # returning repo infos and raw json with all weeks and adds and dels by week timestamp
    raw_git_data = retrive_git_data(owner_repo_names, authorization_token)

    # treats the data to the dates we specifyed a
    #  returns addtions and deletions data frames together with the raw dataframe

    return tretened_df(raw_git_data, start_date_aggregation, end_date_aggregation, year_to_start,year_to_finish ) , raw_git_data



# Extracting data using the compile function

In [5]:
df.shape

(403, 23)

In [7]:
df.columns

Index(['grant_id', 'active_x', 'approved', 'address_x', 'title_x', 'url',
       'description_x', 'created_on_x', 'active_y', 'title_y', 'address_y',
       'amount_received', 'amount_received_in_round', 'contribution_count',
       'contributor_count', 'description_y', 'website', 'github_project_url',
       'twitter_handle_2', 'twitter_handle_1', 'twitter_verified',
       'created_on_y', 'last_update'],
      dtype='object')

In [9]:
sample = df['github_project_url'][0:15]

In [12]:
url_repo_series =  sample
url_repo_series = url_repo_series[~url_repo_series.isna()]

authorization_token = 'YOUR_GITHUB_PAT_HERE'

start_date_aggregation = 36 #  September 7 started the GR15 round
end_date_aggregation = 12 # 6 months back

year_to_start = 2022
year_to_finish = 2022

x1, x2 = repo_additions_deletion(url_repo_series,
                        start_date_aggregation,
                        end_date_aggregation,
                        year_to_start,
                        year_to_finish, 
 
                        authorization_token )

## final dataframes 

- x1 is a object with two dataframes , the first one containing the 'addtions by week' dataframe of each url, and the second one with the same schema but with the deletions information.  
        - As mentioned before : Nan means the repo did not existed yet
        - "0" means it existed but had no deletionos adtion that week

- x2 would be a dataset containing the url / owner_repo / repo_name / repo_data where repodata is a json with all the data obtaned from the API for that url
        - 404 means that the repo does not exist anymore
        - 203 means it never had any interacions, nor additions neither deletions



In [13]:
x1[0]

,2022-03-19,2022-03-26,2022-04-02,2022-04-09,2022-04-16,2022-04-23,2022-04-30,2022-05-07,2022-05-14,2022-05-21,...,2022-07-02,2022-07-09,2022-07-16,2022-07-23,2022-07-30,2022-08-06,2022-08-13,2022-08-20,2022-08-27,2022-09-03
https://github.com/socathie/zkML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,45,0,0
https://github.com/AthanorLabs/atomic-swap,1290,0,11,1153,0,0,120,330,1469,1488,...,108,4,0,0,1740,736,1336,557,194,4388
https://github.com/holic/web3-scaffold,0,0,8,7413,5,0,0,0,0,0,...,0,138,109,19,0,0,0,0,0,0


In [14]:
x1[1]

,2022-03-19,2022-03-26,2022-04-02,2022-04-09,2022-04-16,2022-04-23,2022-04-30,2022-05-07,2022-05-14,2022-05-21,...,2022-07-02,2022-07-09,2022-07-16,2022-07-23,2022-07-30,2022-08-06,2022-08-13,2022-08-20,2022-08-27,2022-09-03
https://github.com/socathie/zkML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,-8,0,0
https://github.com/AthanorLabs/atomic-swap,-246,0,-8,-301,0,0,-18,-138,-1171,-590,...,-77,-264,0,0,-695,-804,-1896,-1261,-6,-1322
https://github.com/holic/web3-scaffold,0,0,-4,-72,-7,0,0,0,0,0,...,0,-12,-23,-4,0,0,0,0,0,0


In [15]:
x2

,owner,repo,extract_status_code,repo_data
url,,,,
https://github.com/socathie/zkML,socathie,zkML,200,"[[1653782400, 70916, -188], [1654387200, 162, ..."
https://github.com/DIMCHERRY/NFT-Ads,DIMCHERRY,NFT-Ads,404,"{'message': 'Not Found', 'documentation_url': ..."
https://github.com/SaveWithBuckets/Buckets_v1,SaveWithBuckets,Buckets_v1,404,"{'message': 'Not Found', 'documentation_url': ..."
https://github.com/pshdev0/dexode,pshdev0,dexode,404,"{'message': 'Not Found', 'documentation_url': ..."
https://github.com/AthanorLabs/atomic-swap,AthanorLabs,atomic-swap,200,"[[1634428800, 140425, -134039], [1635033600, 4..."
https://github.com/holic/web3-scaffold,holic,web3-scaffold,200,"[[1642291200, 59326, 0], [1642896000, 0, 0], [..."


# How to use it and analysis tips:

####  Merge a 'flag' column from the original dataset and use this data to do some data analysis and extract info for some supervised learning algos like:
    -logistic regression on the rejected grants about the most active weeks before the start of the subscription ( analyzing  the dates as features and getting the coefficient  of that weeks) 
#### Some pure data analysis: 
    - overall comparison between activity level between approved and rejected grants
    - distribution of activity given time
#### Unsupervised machine learning:
    -Grouping repos and checking for patterns in there constructions


# Case you get the final function running for too long:

Till now the function has no exception for when the limit rate is exceeded therefore it runs forever, if you think it's taking too long, run this in other notebook:

In [6]:
url = 'https://api.github.com/user/repos'
headers = {'Authorization': 'YOUR_PAT'}
reponse = requests.get(url, headers=headers).headers

Check for 'X-RateLimit-Remaining' and X-RateLimit-Reset', you will get the answer if you have reached your calls limit
and when can your rate limit will be restored

In [7]:
reponse

{'Server': 'GitHub.com', 'Date': 'Sat, 13 May 2023 13:55:01 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '149', 'X-GitHub-Media-Type': 'github.v3; format=json', 'x-github-api-version-selected': '2022-11-28', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '59', 'X-RateLimit-Reset': '1683989701', 'X-RateLimit-Used': '1', 'X-RateLimit-Resource': 'core', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'Content-S